In [0]:
from keras.models import Model
from keras.layers import Input, Activation, merge, Dense, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, add, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator

from keras.datasets import cifar10
from keras.utils import np_utils

import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import toimage

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

Using TensorFlow backend.


170500096/170498071 [==============================] - 16s 0us/step


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import toimage
from sklearn.model_selection import train_test_split
cifar10_labels = np.array([
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'])

In [0]:
# floatに変換しておく
# 正規化するために
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 正規化
X_train /= 255.0
X_test /= 255.0
# RGB各要素について0-255の値で色を表現する、例えば白は（R 255、G 255、B 255）で表現

# one-hot vector に変換したほうが都合が良い
# たとえば 4 ではなくて [0, 0, 0, 0, 1, 0, 0, 0, 0, 0] のようなデータにする
# こうしないとクラス分類ではなくて回帰として扱われてしまいうまくいかない
n_classes = 10
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)

In [0]:
#
# ResNet のモデルを定義するセル
#

# inputs と residual 2つの処理結果を受け取って、両者を足す

# inputs：ショートカットせずに畳み込みやReLuなどをしたあとの信号
# residual：畳み込みなどを通らずショートカットして流れてくる信号のこと、つまりモジュールに入ってくる信号と同じ
def _shortcut(inputs, residual):
  
  # residual のほうのフィルタ数を取得する
  # ちなみにデフォルトでは
  # _keras_shape[1] 画像の幅
  # _keras_shape[2] 画像の高さ
  # _keras_shape[3] チャンネル数
  # チャンネル数、幅、高さの順番のこともあるが、今回はデフォルトでOK
  n_filters = residual._keras_shape[3]
  
  # inputs と residual とでチャネル数が違うかもしれない。
  # そのままだと足せないので、1x1 conv を使って residual 側のフィルタ数に合わせている
  shortcut = Convolution2D(n_filters, (1,1), strides=(1,1),kernel_initializer='glorot_normal', padding='same')(inputs)
 
  # 2つを足す
  return add([shortcut, residual])


# ResBlock を定義
# ここでの処理は BatchNorm → ReLU → Conv とシンプルなものにしてあるが、
# ここを色々変更する改良案が無数にある
###ResBlock とは、ショートカット構造のあるレイヤーの組み合わせ（ブロック）のこと
def _resblock(n_filters, strides=(1,1)):
  def f(inputs):  
    x = Convolution2D(n_filters, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(n_filters, (3,3), strides=strides,kernel_initializer='he_normal', padding='same')(x)    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(n_filters * 4, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # 入力そのものと、BN→ReLU→Conv したものとを足す
    # この部分がResNetのもっとも重要な点
    return _shortcut(inputs, x)
  
  return f

def resnet():
    inputs = Input(shape=(32, 32, 3))

    x = BatchNormalization()(inputs)
    x = Activation('relu')(x)
    x = _resblock(n_filters=64)(x)
#    x = Dropout(0.4)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=64)(x)
    
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=128)(x)
 #   x = Dropout(0.4)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
 #   x = _resblock(n_filters=128)(x)
    
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=256)(x)
#    x = Dropout(0.4)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=256)(x)
  

    x = GlobalAveragePooling2D()(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=x)
    return model
  
model = resnet()
model.compile(optimizer='sgd', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_136 (BatchN (None, 32, 32, 3)    12          input_12[0][0]                   
__________________________________________________________________________________________________
activation_136 (Activation)     (None, 32, 32, 3)    0           batch_normalization_136[0][0]    
__________________________________________________________________________________________________
conv2d_133 (Conv2D)             (None, 32, 32, 64)   256         activation_136[0][0]             
__________________________________________________________________________________________________
batch_norm

In [0]:
#checkpoint
callback_op = ModelCheckpoint(filepath='weights.{epoch:02d}.hdf5')
callback_es = EarlyStopping(monitor='val_acc', patience=10,
                            mode='auto', verbose=1)

batch_size = 50
num_epochs = 100 # 普通は100〜300くらいを指定することが多い

#h = model.fit(X_train, Y_train,
#               batch_size=batch_size,
#               epochs=epochs,
#               validation_split=0.2,
#               callbacks=[callback_op, callback_es])

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train,
                                  batch_size=batch_size),
                                  epochs=num_epochs,
                                  validation_data=(X_test, Y_test),
                                  steps_per_epoch=int(np.ceil(X_train.shape[0] / float(batch_size))),
                                  workers=1)

model.save('model.hdf5')

Epoch 1/100
1000/1000 [==============================] - 128s 128ms/step - loss: 1.8378 - acc: 0.3317 - val_loss: 1.6514 - val_acc: 0.4027
Epoch 2/100
1000/1000 [==============================] - 122s 122ms/step - loss: 1.5766 - acc: 0.4315 - val_loss: 1.5471 - val_acc: 0.4575
Epoch 3/100
1000/1000 [==============================] - 122s 122ms/step - loss: 1.4463 - acc: 0.4786 - val_loss: 1.4587 - val_acc: 0.4782
Epoch 4/100
1000/1000 [==============================] - 122s 122ms/step - loss: 1.3519 - acc: 0.5177 - val_loss: 1.4318 - val_acc: 0.5139
Epoch 5/100
1000/1000 [==============================] - 122s 122ms/step - loss: 1.2770 - acc: 0.5453 - val_loss: 1.2562 - val_acc: 0.5498
Epoch 6/100
1000/1000 [==============================] - 121s 121ms/step - loss: 1.2153 - acc: 0.5706 - val_loss: 1.2200 - val_acc: 0.5792
Epoch 7/100
1000/1000 [==============================] - 121s 121ms/step - loss: 1.1561 - acc: 0.5917 - val_loss: 1.1348 - val_acc: 0.6070
Epoch 8/100
1000/1000 [====

KeyboardInterrupt: ignored

In [0]:
#
# ResNet のモデルを定義するセル
#

# inputs と residual 2つの処理結果を受け取って、両者を足す

# inputs：ショートカットせずに畳み込みやReLuなどをしたあとの信号
# residual：畳み込みなどを通らずショートカットして流れてくる信号のこと、つまりモジュールに入ってくる信号と同じ
def _shortcut(inputs, residual):
  
  # residual のほうのフィルタ数を取得する
  # ちなみにデフォルトでは
  # _keras_shape[1] 画像の幅
  # _keras_shape[2] 画像の高さ
  # _keras_shape[3] チャンネル数
  # チャンネル数、幅、高さの順番のこともあるが、今回はデフォルトでOK
  n_filters = residual._keras_shape[3]
  
  # inputs と residual とでチャネル数が違うかもしれない。
  # そのままだと足せないので、1x1 conv を使って residual 側のフィルタ数に合わせている
  shortcut = Convolution2D(n_filters, (1,1), strides=(1,1),kernel_initializer='glorot_normal', padding='same')(inputs)
 
  # 2つを足す
  return add([shortcut, residual])


# ResBlock を定義
# ここでの処理は BatchNorm → ReLU → Conv とシンプルなものにしてあるが、
# ここを色々変更する改良案が無数にある
###ResBlock とは、ショートカット構造のあるレイヤーの組み合わせ（ブロック）のこと
def _resblock(n_filters, strides=(1,1)):
  def f(inputs):  
    x = Convolution2D(n_filters, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(n_filters, (3,3), strides=strides,kernel_initializer='he_normal', padding='same')(x)    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(n_filters * 4, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # 入力そのものと、BN→ReLU→Conv したものとを足す
    # この部分がResNetのもっとも重要な点
    return _shortcut(inputs, x)
  
  return f

def resnet():
    inputs = Input(shape=(32, 32, 3))

    x = BatchNormalization()(inputs)
    x = Activation('relu')(x)
    x = _resblock(n_filters=64)(x)
#    x = Dropout(0.4)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=64)(x)
    
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=128)(x)
 #   x = Dropout(0.4)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
 #   x = _resblock(n_filters=128)(x)
    
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=256)(x)
#    x = Dropout(0.4)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=256)(x)

    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=512)(x)
#    x = Dropout(0.4)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=256)(x)

  
    x = GlobalAveragePooling2D()(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=x)
    return model
  
model = resnet()
model.compile(optimizer='sgd', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_196 (BatchN (None, 32, 32, 3)    12          input_15[0][0]                   
__________________________________________________________________________________________________
activation_196 (Activation)     (None, 32, 32, 3)    0           batch_normalization_196[0][0]    
__________________________________________________________________________________________________
conv2d_181 (Conv2D)             (None, 32, 32, 64)   256         activation_196[0][0]             
__________________________________________________________________________________________________
batch_norm

In [0]:
#checkpoint
callback_op = ModelCheckpoint(filepath='weights.{epoch:02d}.hdf5')
callback_es = EarlyStopping(monitor='val_acc', patience=10,
                            mode='auto', verbose=1)

batch_size = 50
num_epochs = 100 # 普通は100〜300くらいを指定することが多い

#h = model.fit(X_train, Y_train,
#               batch_size=batch_size,
#               epochs=epochs,
#               validation_split=0.2,
#               callbacks=[callback_op, callback_es])

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train,
                                  batch_size=batch_size),
                                  epochs=num_epochs,
                                  validation_data=(X_test, Y_test),
                                  steps_per_epoch=int(np.ceil(X_train.shape[0] / float(batch_size))),
                                  workers=1)

model.save('model.hdf5')

Epoch 1/100
1000/1000 [==============================] - 168s 168ms/step - loss: 1.8138 - acc: 0.3370 - val_loss: 1.5585 - val_acc: 0.4313
Epoch 2/100
1000/1000 [==============================] - 158s 158ms/step - loss: 1.5245 - acc: 0.4481 - val_loss: 1.4141 - val_acc: 0.4911
Epoch 3/100
1000/1000 [==============================] - 158s 158ms/step - loss: 1.3839 - acc: 0.5003 - val_loss: 1.4309 - val_acc: 0.5073
Epoch 4/100
1000/1000 [==============================] - 157s 157ms/step - loss: 1.2703 - acc: 0.5460 - val_loss: 1.2326 - val_acc: 0.5636
Epoch 5/100
1000/1000 [==============================] - 157s 157ms/step - loss: 1.1884 - acc: 0.5788 - val_loss: 1.1013 - val_acc: 0.6088
Epoch 6/100
1000/1000 [==============================] - 157s 157ms/step - loss: 1.1218 - acc: 0.6002 - val_loss: 1.0437 - val_acc: 0.6333
Epoch 7/100
1000/1000 [==============================] - 157s 157ms/step - loss: 1.0609 - acc: 0.6260 - val_loss: 1.0384 - val_acc: 0.6412
Epoch 8/100
1000/1000 [====

In [0]:
import numpy as np
import matplotlib.pyplot as plt

epoch = np.arange(len(model.history['acc']))

# accuracy と validation accuracy の推移をプロットする
plt.title('Accuracy')
plt.ylim(0.0, 1.0)
plt.plot(epoch, model.history['acc'], label='train')
plt.plot(epoch, model.history['val_acc'], label='validation')
plt.legend()
plt.show()

In [0]:
#
# ResNet のモデルを定義するセル
#

# inputs と residual 2つの処理結果を受け取って、両者を足す

# inputs：ショートカットせずに畳み込みやReLuなどをしたあとの信号
# residual：畳み込みなどを通らずショートカットして流れてくる信号のこと、つまりモジュールに入ってくる信号と同じ
def _shortcut(inputs, residual):
  
  # residual のほうのフィルタ数を取得する
  # ちなみにデフォルトでは
  # _keras_shape[1] 画像の幅
  # _keras_shape[2] 画像の高さ
  # _keras_shape[3] チャンネル数
  # チャンネル数、幅、高さの順番のこともあるが、今回はデフォルトでOK
  n_filters = residual._keras_shape[3]
  
  # inputs と residual とでチャネル数が違うかもしれない。
  # そのままだと足せないので、1x1 conv を使って residual 側のフィルタ数に合わせている
  shortcut = Convolution2D(n_filters, (1,1), strides=(1,1),kernel_initializer='glorot_normal', padding='same')(inputs)
 
  # 2つを足す
  return add([shortcut, residual])


# ResBlock を定義
# ここでの処理は BatchNorm → ReLU → Conv とシンプルなものにしてあるが、
# ここを色々変更する改良案が無数にある
###ResBlock とは、ショートカット構造のあるレイヤーの組み合わせ（ブロック）のこと
def _resblock(n_filters, strides=(1,1)):
  def f(inputs):  
    x = Convolution2D(n_filters, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(n_filters, (3,3), strides=strides,kernel_initializer='he_normal', padding='same')(x)    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(n_filters * 4, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # 入力そのものと、BN→ReLU→Conv したものとを足す
    # この部分がResNetのもっとも重要な点
    return _shortcut(inputs, x)
  
  return f

def resnet():
    inputs = Input(shape=(32, 32, 3))

    x = BatchNormalization()(inputs)
    x = Activation('relu')(x)
    x = _resblock(n_filters=64)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=64)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
#    x = Dropout(0.2)(x)
  
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=128)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=128)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)
    
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=256)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=256)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)

    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=512)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=256)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)
  
    x = GlobalAveragePooling2D()(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=x)
    return model
  
model = resnet()
model.compile(optimizer='sgd', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_77 (BatchNo (None, 32, 32, 3)    12          input_4[0][0]                    
__________________________________________________________________________________________________
activation_77 (Activation)      (None, 32, 32, 3)    0           batch_normalization_77[0][0]     
__________________________________________________________________________________________________
conv2d_65 (Conv2D)              (None, 32, 32, 64)   256         activation_77[0][0]              
__________________________________________________________________________________________________
batch_norm

In [0]:
#
# ResNet のモデルを定義するセル
#

# inputs と residual 2つの処理結果を受け取って、両者を足す

# inputs：ショートカットせずに畳み込みやReLuなどをしたあとの信号
# residual：畳み込みなどを通らずショートカットして流れてくる信号のこと、つまりモジュールに入ってくる信号と同じ
def _shortcut(inputs, residual):
  
  # residual のほうのフィルタ数を取得する
  # ちなみにデフォルトでは
  # _keras_shape[1] 画像の幅
  # _keras_shape[2] 画像の高さ
  # _keras_shape[3] チャンネル数
  # チャンネル数、幅、高さの順番のこともあるが、今回はデフォルトでOK
  n_filters = residual._keras_shape[3]
  
  # inputs と residual とでチャネル数が違うかもしれない。
  # そのままだと足せないので、1x1 conv を使って residual 側のフィルタ数に合わせている
  shortcut = Convolution2D(n_filters, (1,1), strides=(1,1),kernel_initializer='glorot_normal', padding='same')(inputs)
 
  # 2つを足す
  return add([shortcut, residual])


# ResBlock を定義
# ここでの処理は BatchNorm → ReLU → Conv とシンプルなものにしてあるが、
# ここを色々変更する改良案が無数にある
###ResBlock とは、ショートカット構造のあるレイヤーの組み合わせ（ブロック）のこと
def _resblock(n_filters, strides=(1,1)):
  def f(inputs):  
    x = Convolution2D(n_filters, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(n_filters, (3,3), strides=strides,kernel_initializer='he_normal', padding='same')(x)    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(n_filters * 4, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # 入力そのものと、BN→ReLU→Conv したものとを足す
    # この部分がResNetのもっとも重要な点
    return _shortcut(inputs, x)
  
  return f

def resnet():
    inputs = Input(shape=(32, 32, 3))

    x = BatchNormalization()(inputs)
    x = Activation('relu')(x)
    x = _resblock(n_filters=64)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=64)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
#    x = Dropout(0.2)(x)
  
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=128)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=128)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)
    
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=256)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=256)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)

    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=512)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=256)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)
  
    x = GlobalAveragePooling2D()(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=x)
    return model
  
model = resnet()
model.compile(optimizer='sgd', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [0]:
#checkpoint
#callback_op = ModelCheckpoint(filepath='weights.{epoch:02d}.hdf5')
callback_es = EarlyStopping(monitor='val_acc', patience=10,
                            mode='auto', verbose=1)

batch_size = 30
num_epochs = 100 # 普通は100〜300くらいを指定することが多い

#h = model.fit(X_train, Y_train,
#               batch_size=batch_size,
#               epochs=epochs,
#               validation_split=0.2,
#               callbacks=[callback_op, callback_es])

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train,
                                  batch_size=batch_size),
                                  epochs=num_epochs,
                                  validation_data=(X_test, Y_test),
                                  steps_per_epoch=int(np.ceil(X_train.shape[0] / float(batch_size))),
                                  workers=1)

model.save('model.hdf5')

Epoch 1/100
1667/1667 [==============================] - 213s 128ms/step - loss: 1.7671 - acc: 0.3513 - val_loss: 1.6437 - val_acc: 0.4352
Epoch 2/100
1667/1667 [==============================] - 207s 124ms/step - loss: 1.4559 - acc: 0.4759 - val_loss: 1.3901 - val_acc: 0.5157
Epoch 3/100
1667/1667 [==============================] - 206s 124ms/step - loss: 1.2916 - acc: 0.5388 - val_loss: 1.2000 - val_acc: 0.5794
Epoch 4/100
1667/1667 [==============================] - 206s 124ms/step - loss: 1.1803 - acc: 0.5790 - val_loss: 1.1703 - val_acc: 0.5924
Epoch 5/100
1667/1667 [==============================] - 206s 124ms/step - loss: 1.0930 - acc: 0.6120 - val_loss: 1.0443 - val_acc: 0.6483
Epoch 6/100
1667/1667 [==============================] - 206s 124ms/step - loss: 1.0208 - acc: 0.6409 - val_loss: 0.9213 - val_acc: 0.6759
Epoch 7/100
1667/1667 [==============================] - 206s 124ms/step - loss: 0.9643 - acc: 0.6611 - val_loss: 0.9968 - val_acc: 0.6620
Epoch 8/100
1667/1667 [====

In [0]:
import numpy as np
import matplotlib.pyplot as plt

epoch = np.arange(len(h.history['acc']))

# accuracy と validation accuracy の推移をプロットする
plt.title('Accuracy')
plt.ylim(0.0, 1.0)
plt.plot(epoch, h.history['acc'], label='train')
plt.plot(epoch, h.history['val_acc'], label='validation')
plt.legend()
plt.show()

In [0]:
#
# ResNet のモデルを定義するセル
#

# inputs と residual 2つの処理結果を受け取って、両者を足す

# inputs：ショートカットせずに畳み込みやReLuなどをしたあとの信号
# residual：畳み込みなどを通らずショートカットして流れてくる信号のこと、つまりモジュールに入ってくる信号と同じ
def _shortcut(inputs, residual):
  
  # residual のほうのフィルタ数を取得する
  # ちなみにデフォルトでは
  # _keras_shape[1] 画像の幅
  # _keras_shape[2] 画像の高さ
  # _keras_shape[3] チャンネル数
  # チャンネル数、幅、高さの順番のこともあるが、今回はデフォルトでOK
  n_filters = residual._keras_shape[3]
  
  # inputs と residual とでチャネル数が違うかもしれない。
  # そのままだと足せないので、1x1 conv を使って residual 側のフィルタ数に合わせている
  shortcut = Convolution2D(n_filters, (1,1), strides=(1,1),kernel_initializer='glorot_normal', padding='same')(inputs)
 
  # 2つを足す
  return add([shortcut, residual])


# ResBlock を定義
# ここでの処理は BatchNorm → ReLU → Conv とシンプルなものにしてあるが、
# ここを色々変更する改良案が無数にある
###ResBlock とは、ショートカット構造のあるレイヤーの組み合わせ（ブロック）のこと
def _resblock(n_filters, strides=(1,1)):
  def f(inputs):  
    
    x = BatchNormalization()(inputs)
    x = Activation('relu')(x)
    
    x = Convolution2D(n_filters, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(n_filters, (3,3), strides=strides,kernel_initializer='he_normal', padding='same')(x)    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(n_filters * 4, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
  #  x = Dropout(0.3)(x)
    
    # 入力そのものと、BN→ReLU→Conv したものとを足す
    # この部分がResNetのもっとも重要な点
    return _shortcut(inputs, x)
  
  return f

def resnet():
    inputs = Input(shape=(32, 32, 3))

    x = BatchNormalization()(inputs)
    x = Activation('relu')(x)
    x = _resblock(n_filters=64)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=64)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
#    x = Dropout(0.2)(x)
  
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=128)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=128)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)
    
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=256)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=256)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)

    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=512)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=256)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)
  
    x = GlobalAveragePooling2D()(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=x)
    return model
  
model = resnet()
model.compile(optimizer='sgd', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_41 (BatchNo (None, 32, 32, 3)    12          input_3[0][0]                    
__________________________________________________________________________________________________
activation_41 (Activation)      (None, 32, 32, 3)    0           batch_normalization_41[0][0]     
__________________________________________________________________________________________________
batch_normalization_42 (BatchNo (None, 32, 32, 3)    12          activation_41[0][0]              
__________________________________________________________________________________________________
activation

In [0]:
#checkpoint
#callback_op = ModelCheckpoint(filepath='weights.{epoch:02d}.hdf5')
callback_es = EarlyStopping(monitor='val_acc', patience=10,
                            mode='auto', verbose=1)

batch_size = 30
num_epochs = 100 # 普通は100〜300くらいを指定することが多い

#h = model.fit(X_train, Y_train,
#               batch_size=batch_size,
#               epochs=epochs,
#               validation_split=0.2,
#               callbacks=[callback_op, callback_es])

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train,
                                  batch_size=batch_size),
                                  epochs=num_epochs,
                                  validation_data=(X_test, Y_test),
                                  steps_per_epoch=int(np.ceil(X_train.shape[0] / float(batch_size))),
                                  workers=1)

model.save('model.hdf5')

Epoch 1/100
1667/1667 [==============================] - 219s 131ms/step - loss: 1.8476 - acc: 0.3225 - val_loss: 1.5846 - val_acc: 0.4303
Epoch 2/100
1667/1667 [==============================] - 214s 128ms/step - loss: 1.5402 - acc: 0.4415 - val_loss: 1.3956 - val_acc: 0.4992
Epoch 3/100
1667/1667 [==============================] - 214s 129ms/step - loss: 1.3754 - acc: 0.5067 - val_loss: 1.3202 - val_acc: 0.5406
Epoch 4/100
1667/1667 [==============================] - 215s 129ms/step - loss: 1.2529 - acc: 0.5548 - val_loss: 1.1222 - val_acc: 0.5982
Epoch 5/100
1667/1667 [==============================] - 213s 128ms/step - loss: 1.1626 - acc: 0.5866 - val_loss: 1.1120 - val_acc: 0.6103
Epoch 6/100
1667/1667 [==============================] - 215s 129ms/step - loss: 1.0827 - acc: 0.6173 - val_loss: 1.0794 - val_acc: 0.6338
Epoch 7/100
1667/1667 [==============================] - 216s 129ms/step - loss: 1.0187 - acc: 0.6395 - val_loss: 1.0293 - val_acc: 0.6502
Epoch 8/100
1667/1667 [====

オプティマイザを変更したら精度向上

opt = Adam(amsgrad=True)  
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
#
# ResNet のモデルを定義するセル
#

# inputs と residual 2つの処理結果を受け取って、両者を足す

# inputs：ショートカットせずに畳み込みやReLuなどをしたあとの信号
# residual：畳み込みなどを通らずショートカットして流れてくる信号のこと、つまりモジュールに入ってくる信号と同じ
def _shortcut(inputs, residual):
  
  # residual のほうのフィルタ数を取得する
  # ちなみにデフォルトでは
  # _keras_shape[1] 画像の幅
  # _keras_shape[2] 画像の高さ
  # _keras_shape[3] チャンネル数
  # チャンネル数、幅、高さの順番のこともあるが、今回はデフォルトでOK
  n_filters = residual._keras_shape[3]
  
  # inputs と residual とでチャネル数が違うかもしれない。
  # そのままだと足せないので、1x1 conv を使って residual 側のフィルタ数に合わせている
  shortcut = Convolution2D(n_filters, (1,1), strides=(1,1),kernel_initializer='glorot_normal', padding='same')(inputs)
 
  # 2つを足す
  return add([shortcut, residual])


# ResBlock を定義
# ここでの処理は BatchNorm → ReLU → Conv とシンプルなものにしてあるが、
# ここを色々変更する改良案が無数にある
###ResBlock とは、ショートカット構造のあるレイヤーの組み合わせ（ブロック）のこと
def _resblock(n_filters, strides=(1,1)):
  def f(inputs):  
    x = Convolution2D(n_filters, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(n_filters, (3,3), strides=strides,kernel_initializer='he_normal', padding='same')(x)    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(n_filters * 4, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # 入力そのものと、BN→ReLU→Conv したものとを足す
    # この部分がResNetのもっとも重要な点
    return _shortcut(inputs, x)
  
  return f

def resnet():
    inputs = Input(shape=(32, 32, 3))

    x = BatchNormalization()(inputs)
    x = Activation('relu')(x)
    x = _resblock(n_filters=64)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=64)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
#    x = Dropout(0.2)(x)
  
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=128)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=128)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)
    
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=256)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=256)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)

    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=512)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=256)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)
  
    x = GlobalAveragePooling2D()(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=x)
    return model
  
opt = Adam(amsgrad=True)  

model = resnet()
#model.compile(optimizer='sgd', loss='categorical_crossentropy',
model.compile(optimizer=opt, loss='categorical_crossentropy',
              metrics=['accuracy'])

#model.summary()

#checkpoint
#callback_op = ModelCheckpoint(filepath='weights.{epoch:02d}.hdf5')
callback_es = EarlyStopping(monitor='val_acc', patience=10,
                            mode='auto', verbose=1)

batch_size = 30
num_epochs = 100 # 普通は100〜300くらいを指定することが多い

#h = model.fit(X_train, Y_train,
#               batch_size=batch_size,
#               epochs=epochs,
#               validation_split=0.2,
#               callbacks=[callback_op, callback_es])

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train,
                                  batch_size=batch_size),
                                  epochs=num_epochs,
                                  validation_data=(X_test, Y_test),
                                  steps_per_epoch=int(np.ceil(X_train.shape[0] / float(batch_size))),
                                  workers=1)

model.save('model.hdf5')

Epoch 1/100
1667/1667 [==============================] - 257s 154ms/step - loss: 1.6481 - acc: 0.4219 - val_loss: 1.4197 - val_acc: 0.5307
Epoch 2/100
1667/1667 [==============================] - 250s 150ms/step - loss: 1.1563 - acc: 0.5942 - val_loss: 1.0373 - val_acc: 0.6553
Epoch 3/100
1667/1667 [==============================] - 250s 150ms/step - loss: 0.9009 - acc: 0.6872 - val_loss: 0.7827 - val_acc: 0.7380
Epoch 4/100
1667/1667 [==============================] - 250s 150ms/step - loss: 0.7595 - acc: 0.7369 - val_loss: 0.8198 - val_acc: 0.7346
Epoch 5/100
1667/1667 [==============================] - 250s 150ms/step - loss: 0.6588 - acc: 0.7734 - val_loss: 0.5847 - val_acc: 0.8029
Epoch 6/100
1667/1667 [==============================] - 249s 150ms/step - loss: 0.5944 - acc: 0.7939 - val_loss: 0.5331 - val_acc: 0.8170
Epoch 7/100
1667/1667 [==============================] - 250s 150ms/step - loss: 0.5401 - acc: 0.8163 - val_loss: 0.7370 - val_acc: 0.7746
Epoch 8/100
1667/1667 [====

Data Augumentation のパラメータを変更するも制度は向上せず

        rotation_range=45,  
        width_shift_range=0.4, 
        height_shift_range=0.4,  

In [0]:
#
# ResNet のモデルを定義するセル
#

# inputs と residual 2つの処理結果を受け取って、両者を足す

# inputs：ショートカットせずに畳み込みやReLuなどをしたあとの信号
# residual：畳み込みなどを通らずショートカットして流れてくる信号のこと、つまりモジュールに入ってくる信号と同じ
def _shortcut(inputs, residual):
  
  # residual のほうのフィルタ数を取得する
  # ちなみにデフォルトでは
  # _keras_shape[1] 画像の幅
  # _keras_shape[2] 画像の高さ
  # _keras_shape[3] チャンネル数
  # チャンネル数、幅、高さの順番のこともあるが、今回はデフォルトでOK
  n_filters = residual._keras_shape[3]
  
  # inputs と residual とでチャネル数が違うかもしれない。
  # そのままだと足せないので、1x1 conv を使って residual 側のフィルタ数に合わせている
  shortcut = Convolution2D(n_filters, (1,1), strides=(1,1),kernel_initializer='glorot_normal', padding='same')(inputs)
 
  # 2つを足す
  return add([shortcut, residual])


# ResBlock を定義
# ここでの処理は BatchNorm → ReLU → Conv とシンプルなものにしてあるが、
# ここを色々変更する改良案が無数にある
###ResBlock とは、ショートカット構造のあるレイヤーの組み合わせ（ブロック）のこと
def _resblock(n_filters, strides=(1,1)):
  def f(inputs):  
    weight_decay = 0.01
#    x = Convolution2D(n_filters, (1,1), strides=strides,kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(weight_decay), activity_regularizer=regularizers.l1(weight_decay))(inputs)
    x = Convolution2D(n_filters, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

#    x = Convolution2D(n_filters, (3,3), strides=strides,kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(weight_decay), activity_regularizer=regularizers.l1(weight_decay))(x)    
    x = Convolution2D(n_filters, (3,3), strides=strides,kernel_initializer='he_normal', padding='same')(x)    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

#    x = Convolution2D(n_filters * 4, (1,1), strides=strides,kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(weight_decay), activity_regularizer=regularizers.l1(weight_decay))(x)
    x = Convolution2D(n_filters * 4, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # 入力そのものと、BN→ReLU→Conv したものとを足す
    # この部分がResNetのもっとも重要な点
    return _shortcut(inputs, x)
  
  return f

def resnet():
    inputs = Input(shape=(32, 32, 3))

    x = BatchNormalization()(inputs)
    x = Activation('relu')(x)
    x = _resblock(n_filters=64)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=64)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
#    x = Dropout(0.2)(x)
  
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=128)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=128)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)
    
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=256)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=256)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)

    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=512)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=256)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)
  
    x = GlobalAveragePooling2D()(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=x)
    return model

model = resnet()
opt = Adam(amsgrad=True)  
#model.compile(optimizer='sgd', loss='categorical_crossentropy',
model.compile(optimizer=opt, loss='categorical_crossentropy',
              metrics=['accuracy'])

#model.summary()

#checkpoint
#callback_op = ModelCheckpoint(filepath='weights.{epoch:02d}.hdf5')
callback_es = EarlyStopping(monitor='val_acc', patience=10,
                            mode='auto', verbose=1)

batch_size = 30
num_epochs = 100 # 普通は100〜300くらいを指定することが多い

#h = model.fit(X_train, Y_train,
#               batch_size=batch_size,
#               epochs=epochs,
#               validation_split=0.2,
#               callbacks=[callback_op, callback_es])

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=45,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.4,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.4,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train,
                                  batch_size=batch_size),
                                  epochs=num_epochs,
                                  validation_data=(X_test, Y_test),
                                  steps_per_epoch=int(np.ceil(X_train.shape[0] / float(batch_size))),
                                  workers=1)

model.save('model.hdf5')

Epoch 1/100
1667/1667 [==============================] - 261s 157ms/step - loss: 1.8426 - acc: 0.3439 - val_loss: 1.8715 - val_acc: 0.3542
Epoch 2/100
1667/1667 [==============================] - 253s 152ms/step - loss: 1.5227 - acc: 0.4557 - val_loss: 1.3784 - val_acc: 0.5096
Epoch 3/100
1667/1667 [==============================] - 253s 152ms/step - loss: 1.2881 - acc: 0.5436 - val_loss: 1.3553 - val_acc: 0.5642
Epoch 4/100
1667/1667 [==============================] - 267s 160ms/step - loss: 1.1222 - acc: 0.6068 - val_loss: 1.0625 - val_acc: 0.6636
Epoch 5/100
1667/1667 [==============================] - 255s 153ms/step - loss: 1.0121 - acc: 0.6462 - val_loss: 0.9310 - val_acc: 0.6917
Epoch 6/100
1667/1667 [==============================] - 253s 152ms/step - loss: 0.9238 - acc: 0.6779 - val_loss: 0.8341 - val_acc: 0.7267
Epoch 7/100
1667/1667 [==============================] - 254s 152ms/step - loss: 0.8598 - acc: 0.7058 - val_loss: 0.8574 - val_acc: 0.7291
Epoch 8/100
1667/1667 [====

KeyboardInterrupt: ignored

weight_decay = 0.01
kernel_regularizer=regularizers.l2(weight_decay), activity_regularizer=regularizers.l1(weight_decay)

上記の正則化はあまりこうかなし

↓_resblockの後の Batch Normalization と Activation の削除も精度が悪化したので、
data augumentation のパラメータを調整し再実行




In [0]:
#
# ResNet のモデルを定義するセル
#

# inputs と residual 2つの処理結果を受け取って、両者を足す

# inputs：ショートカットせずに畳み込みやReLuなどをしたあとの信号
# residual：畳み込みなどを通らずショートカットして流れてくる信号のこと、つまりモジュールに入ってくる信号と同じ
def _shortcut(inputs, residual):
  
  # residual のほうのフィルタ数を取得する
  # ちなみにデフォルトでは
  # _keras_shape[1] 画像の幅
  # _keras_shape[2] 画像の高さ
  # _keras_shape[3] チャンネル数
  # チャンネル数、幅、高さの順番のこともあるが、今回はデフォルトでOK
  n_filters = residual._keras_shape[3]
  
  # inputs と residual とでチャネル数が違うかもしれない。
  # そのままだと足せないので、1x1 conv を使って residual 側のフィルタ数に合わせている
  shortcut = Convolution2D(n_filters, (1,1), strides=(1,1),kernel_initializer='glorot_normal', padding='same')(inputs)
 
  # 2つを足す
  return add([shortcut, residual])


# ResBlock を定義
# ここでの処理は BatchNorm → ReLU → Conv とシンプルなものにしてあるが、
# ここを色々変更する改良案が無数にある
###ResBlock とは、ショートカット構造のあるレイヤーの組み合わせ（ブロック）のこと
def _resblock(n_filters, strides=(1,1)):
  def f(inputs):  
    x = Convolution2D(n_filters, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(n_filters, (3,3), strides=strides,kernel_initializer='he_normal', padding='same')(x)    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(n_filters * 4, (1,1), strides=strides,kernel_initializer='he_normal', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # 入力そのものと、BN→ReLU→Conv したものとを足す
    # この部分がResNetのもっとも重要な点
    return _shortcut(inputs, x)
  
  return f

def resnet():
    inputs = Input(shape=(32, 32, 3))

    x = BatchNormalization()(inputs)
    x = Activation('relu')(x)
    x = _resblock(n_filters=64)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=64)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
#    x = Dropout(0.2)(x)
  
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=128)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=128)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)
    
    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=256)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=256)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)

    x = MaxPooling2D(strides=(2,2))(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = _resblock(n_filters=512)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
#    x = _resblock(n_filters=256)(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
    #    x = Dropout(0.2)(x)
  
    x = GlobalAveragePooling2D()(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=x)
    return model
  
opt = Adam(amsgrad=True)  

model = resnet()
#model.compile(optimizer='sgd', loss='categorical_crossentropy',
model.compile(optimizer=opt, loss='categorical_crossentropy',
              metrics=['accuracy'])

#model.summary()

#checkpoint
#callback_op = ModelCheckpoint(filepath='weights.{epoch:02d}.hdf5')
callback_es = EarlyStopping(monitor='val_acc', patience=10,
                            mode='auto', verbose=1)

batch_size = 30
num_epochs = 100 # 普通は100〜300くらいを指定することが多い

#h = model.fit(X_train, Y_train,
#               batch_size=batch_size,
#               epochs=epochs,
#               validation_split=0.2,
#               callbacks=[callback_op, callback_es])

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest',
        vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train,
                                  batch_size=batch_size),
                                  epochs=num_epochs,
                                  validation_data=(X_test, Y_test),
                                  steps_per_epoch=int(np.ceil(X_train.shape[0] / float(batch_size))),
                                  workers=1)

model.save('model.hdf5')

Epoch 1/100
1667/1667 [==============================] - 317s 190ms/step - loss: 1.7322 - acc: 0.3796 - val_loss: 1.4686 - val_acc: 0.4938
Epoch 2/100
1667/1667 [==============================] - 306s 184ms/step - loss: 1.3385 - acc: 0.5235 - val_loss: 1.4442 - val_acc: 0.5270
Epoch 3/100
1667/1667 [==============================] - 307s 184ms/step - loss: 1.0915 - acc: 0.6191 - val_loss: 0.9541 - val_acc: 0.6823
Epoch 4/100
1667/1667 [==============================] - 306s 183ms/step - loss: 0.9347 - acc: 0.6749 - val_loss: 0.8924 - val_acc: 0.7069
Epoch 5/100
1667/1667 [==============================] - 305s 183ms/step - loss: 0.8354 - acc: 0.7102 - val_loss: 0.7936 - val_acc: 0.7412
Epoch 6/100
1667/1667 [==============================] - 307s 184ms/step - loss: 0.7644 - acc: 0.7353 - val_loss: 0.8018 - val_acc: 0.7489
Epoch 7/100
1667/1667 [==============================] - 307s 184ms/step - loss: 0.6996 - acc: 0.7595 - val_loss: 0.6804 - val_acc: 0.7816
Epoch 8/100
1667/1667 [====